<a href="https://colab.research.google.com/github/riskyhomo/Material-Discovery/blob/main/GANs_for_MD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("/content/100.csv")
df

,Unnamed: 0,Unnamed: 0.1,Composition,MaterialId,Reduced Formula,Elements,NSites,Volume,Density,Point Group,...,Crystal System,Corrected Energy,Formation Energy Per Atom,Decomposition Energy Per Atom,Dimensionality Cheon,Bandgap,Is Train,Decomposition Energy Per Atom All,Decomposition Energy Per Atom MP,Decomposition Energy Per Atom MP OQMD
0,0,0,209231,0,39871,163207,10,302.9202,3.2834,10,...,2,-70.4155,-1.9058,-0.1625,5,0.0023,1.0,0.0,-0.1625,-0.0733
1,1,1,355560,1,220471,181188,16,430.2851,6.8040,8,...,5,-94.0804,-0.5004,-0.0137,3,0.0022,1.0,0.0,-0.0137,-0.0106
2,2,2,173457,2,143223,83526,36,828.6647,12.7292,8,...,5,-237.6016,-1.2960,-0.1195,3,inf,1.0,0.0,-0.1195,-0.1185
3,3,3,344245,3,364467,102592,17,407.2614,8.6776,0,...,5,-93.6931,-0.3064,-0.0837,3,0.0011,1.0,-0.0,-0.0837,-0.0772
4,4,4,254904,4,78813,176755,36,727.5283,6.9851,8,...,5,-254.0953,-0.8718,-0.0186,3,inf,1.0,-0.0,-0.0186,-0.0176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,95,332342,95,142017,181587,21,326.9231,8.6819,13,...,6,-155.9796,-0.9409,-0.0286,3,0.0042,1.0,0.0,-0.0286,-0.0179
96,96,96,111270,96,35982,30932,28,279.1767,3.7338,9,...,2,-113.3359,-0.1675,-0.0114,5,0.0031,1.0,0.0,-0.0114,-0.0093
97,97,97,69830,97,369497,205108,34,626.6744,9.2638,27,...,2,-245.9219,-0.7224,-0.0197,3,0.0004,1.0,0.0,-0.0197,-0.0166
98,98,98,274981,98,174276,165670,16,371.3406,6.5559,30,...,3,-93.6479,-0.8241,-0.0302,3,0.0035,1.0,-0.0,-0.0302,-0.0301


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 22 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Unnamed: 0                             100 non-null    int64  
 1   Unnamed: 0.1                           100 non-null    int64  
 2   Composition                            100 non-null    int64  
 3   MaterialId                             100 non-null    int64  
 4   Reduced Formula                        100 non-null    int64  
 5   Elements                               100 non-null    int64  
 6   NSites                                 100 non-null    int64  
 7   Volume                                 100 non-null    float64
 8   Density                                100 non-null    float64
 9   Point Group                            100 non-null    int64  
 10  Space Group                            100 non-null    int64  
 11  Space G

In [5]:
df.shape

(100, 22)

In [6]:
latent_dim = 100

In [7]:
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, ReLU,LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from keras.layers import Reshape

In [8]:
generator = Sequential([
    Dense( 16,input_dim = latent_dim),
    LeakyReLU(0.2),
    BatchNormalization(),
    Dense(128),
    LeakyReLU(0.2),
    BatchNormalization()
])

In [9]:
generator.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                1616      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 16)                0         
                                                                 
 batch_normalization (Batch  (None, 16)                64        
 Normalization)                                                  
                                                                 
 dense_1 (Dense)             (None, 128)               2176      
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 128)               0         
                                                                 
 batch_normalization_1 (Bat  (None, 128)               512       
 chNormalization)                                       

In [10]:
degenerator = Sequential([
    Input(shape=(128,1)),
    Dense(128, activation=LeakyReLU(0.2)),
    Dense(16, activation=LeakyReLU(0.2)),
    Dense(1, activation="sigmoid")
])

In [11]:
degenerator.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 128, 128)          256       
                                                                 
 dense_3 (Dense)             (None, 128, 16)           2064      
                                                                 
 dense_4 (Dense)             (None, 128, 1)            17        
                                                                 
Total params: 2337 (9.13 KB)
Trainable params: 2337 (9.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
degenerator.trainable=False
gan_input = Input(shape=(latent_dim,))
gan_output = degenerator(generator(gan_input))
gan = Model(gan_input, gan_output)

degenerator.compile(loss="mean_squared_error", optimizer=Adam(learning_rate=0.01, beta_1=0.5))
gan.compile(loss="mean_squared_error", optimizer = Adam(learning_rate=0.01, beta_1=0.5))

In [13]:
gan.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 128)               4368      
                                                                 
 sequential_1 (Sequential)   (None, 128, 1)            2337      
                                                                 
Total params: 6705 (26.19 KB)
Trainable params: 4080 (15.94 KB)
Non-trainable params: 2625 (10.25 KB)
_________________________________________________________________


In [14]:
latent_dim = 100
input_shape = (df.shape[1], 1)
batch_size = 2048
epochs = 100


In [17]:
for epoch in range(epochs):
    # Generate real and fake samples
    idx = np.random.randint(0, 99, size=batch_size)
    real_samples = compositions_scaled[idx]
    noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
    fake_samples = generator.predict(noise)
    fake_samples = np.mean(fake_samples, axis=1)
    fake_samples= np.expand_dims(fake_samples, axis=1)

    # Train the discriminator
    X = np.concatenate((real_samples, fake_samples), axis=0)
    y = np.concatenate((np.ones((batch_size, 1)), np.zeros((batch_size, 1))), axis=0)

    discriminator_loss = degenerator.train_on_batch(X, y)

    # Train the generator
    noise = np.random.normal(0, 1, size=(batch_size, latent_dim))
    y_gan = np.ones(batch_size)
    generator_loss = gan.train_on_batch(noise, y_gan)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}/{epochs}, Discriminator Loss: {discriminator_loss}, Generator Loss: {generator_loss}")

64/64 [==============================] - 0s 2ms/step
Epoch 0/100, Discriminator Loss: 0.2511749863624573, Generator Loss: 0.249396413564682
64/64 [==============================] - 0s 2ms/step
Epoch 10/100, Discriminator Loss: 0.2515120208263397, Generator Loss: 0.24860818684101105
64/64 [==============================] - 0s 3ms/step
Epoch 20/100, Discriminator Loss: 0.2519068717956543, Generator Loss: 0.2478158324956894
64/64 [==============================] - 0s 2ms/step
Epoch 30/100, Discriminator Loss: 0.25240078568458557, Generator Loss: 0.24702435731887817
64/64 [==============================] - 0s 2ms/step
Epoch 40/100, Discriminator Loss: 0.25300896167755127, Generator Loss: 0.24623051285743713
64/64 [==============================] - 0s 2ms/step
Epoch 50/100, Discriminator Loss: 0.25369060039520264, Generator Loss: 0.24543406069278717
64/64 [==============================] - 0s 3ms/step
Epoch 60/100, Discriminator Loss: 0.25440388917922974, Generator Loss: 0.2446400374174118


In [15]:
from sklearn.preprocessing  import MinMaxScaler
scaler = MinMaxScaler()
composition= df["Composition"].to_numpy().reshape(-1,1)
compositions_scaled = scaler.fit_transform(composition)

In [24]:
num_materials = 10
noise = np.random.normal(0, 1, size=(num_materials, latent_dim))
new_compositions_encoded = generator.predict(noise)
new_compositions = scaler.inverse_transform(new_compositions_encoded)

# Print the generated materials
# Print the generated materials
for composition in new_compositions:
    composition_str = ' '.join(map(str, composition))
    print(composition_str)

1/1 [==============================] - 0s 32ms/step
321.56442 -509262.0 -638686.5 -676960.8 -635806.6 -778879.75 -550241.75 -495485.4 -261577.5 -509710.6 -496459.62 -405225.88 -150820.4 -723797.25 -37815.812 -615014.4 -679621.25 -235353.62 -595649.06 -694944.2 -479646.78 294191.06 -490357.62 -838662.4 98541.88 -749178.2 104240.84 -49850.51 9662.691 -684895.06 19364.16 -492088.62 -426068.16 -550753.7 -651367.2 -845750.75 -716021.94 210040.33 -704576.75 -677978.44 -525193.56 -659198.2 -576010.75 -623066.06 447515.7 -345673.6 115121.56 -48166.336 76755.8 -837870.4 -529263.0 313340.3 303794.7 315501.56 -580433.7 -461212.62 -491416.2 273958.25 -653641.56 -665280.6 -625646.1 -634184.4 156941.62 -462175.9 -702232.7 -508633.22 -6721.7383 -208528.67 -685728.06 -544941.2 -543091.1 -298459.84 39608.473 -580937.06 -504733.12 208101.06 -209770.22 -578396.9 -723555.2 -105064.87 -474223.1 -538385.25 329677.5 -600652.44 3568.0662 -168216.77 -371569.84 -294907.34 698974.0 -447521.22 -551995.8 -514762.9

In [52]:
num_materials = 40
noise = np.random.normal(0, 1, size=(num_materials, latent_dim))
new_compositions_encoded = generator.predict(noise)
new_compositions = scaler.inverse_transform(new_compositions_encoded)

# Print the generated materials
for composition in new_compositions:
    print(' '.join(str(composition[0]).split()))

2/2 [==============================] - 0s 12ms/step
-755129.5
-511175.84
-832554.2
-1033627.6
-480673.06
-231912.4
-835523.2
-568963.4
-592738.1
-506165.16
-869467.8
-540641.6
-738649.0
-642757.6
-609090.7
-635676.06
-861195.75
-752086.6
-758880.6
-396475.3
-864726.7
-818951.2
167105.94
-659189.06
-710965.06
-663689.7
-704907.0
90804.91
-577026.3
-960627.4
-554761.5
-493127.88
-826712.6
-715632.9
-1060184.4
-410091.34
-580916.5
-788005.44
-1106838.8
-665478.0


In [27]:
!pip install chempy -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.4/210.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 37.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 kB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.2/101.2 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.7 MB/s eta 0:00:00


In [55]:
import numpy as np

element_symbols = {
    0: 'H', 1: 'He', 2: 'Li', 3: 'Be', 4: 'B', 5: 'C', 6: 'N', 7: 'O', 8: 'F', 9: 'Ne',
    10: 'Na', 11: 'Mg', 12: 'Al', 13: 'Si', 14: 'P', 15: 'S', 16: 'Cl', 17: 'Ar', 18: 'K', 19: 'Ca',
    20: 'Sc', 21: 'Ti', 22: 'V', 23: 'Cr', 24: 'Mn', 25: 'Fe', 26: 'Co', 27: 'Ni', 28: 'Cu', 29: 'Zn',
    30: 'Ga', 31: 'Ge', 32: 'As', 33: 'Se', 34: 'Br', 35: 'Kr', 36: 'Rb', 37: 'Sr', 38: 'Y', 39: 'Zr',
    40: 'Nb', 41: 'Mo', 42: 'Tc', 43: 'Ru', 44: 'Rh', 45: 'Pd', 46: 'Ag', 47: 'Cd', 48: 'In', 49: 'Sn',
    50: 'Sb', 51: 'Te', 52: 'I', 53: 'Xe', 54: 'Cs', 55: 'Ba', 56: 'La', 57: 'Ce', 58: 'Pr', 59: 'Nd',
    60: 'Pm', 61: 'Sm', 62: 'Eu', 63: 'Gd', 64: 'Tb', 65: 'Dy', 66: 'Ho', 67: 'Er', 68: 'Tm', 69: 'Yb',
    70: 'Lu', 71: 'Hf', 72: 'Ta', 73: 'W', 74: 'Re', 75: 'Os', 76: 'Ir', 77: 'Pt', 78: 'Au', 79: 'Hg',
    80: 'Tl', 81: 'Pb', 82: 'Bi', 83: 'Po', 84: 'At', 85: 'Rn', 86: 'Fr', 87: 'Ra', 88: 'Ac', 89: 'Th',
    90: 'Pa', 91: 'U', 92: 'Np', 93: 'Pu', 94: 'Am', 95: 'Cm', 96: 'Bk', 97: 'Cf', 98: 'Es', 99: 'Fm', 100 :'Md', 101 :'Unn' , 102: 'No', 103 :'Lr', 104:'Rf', 105:'Db', 106:'Sg', 107:'Bh', 108:'Hs', 109:'Mt', 110:'Ds', 111:'Rg', 112:'Cn', 113:'Nh', 114:'Fl', 115:'Lv', 116:'Ts', 117:'Og'
}


def composition_to_formula(composition):
    formula = ''
    for i, percentage in enumerate(composition):
        if percentage > 0:
            symbol = element_symbols.get(i, 'Unknown')
            formula += f'{symbol}{int(percentage)}'
    return formula


for composition in new_compositions:
    formula = composition_to_formula(composition)
    print(formula)


Ni206136Rb332840Zr660128In393795Ce67432Nd47301Ho336744Os157232Bi33029Ac615822Lr101048Fl673187Unknown63302Unknown12922Unknown539786
Tc8645Pr30149Bi237057
Ho68368Unknown528076
As455568Br732654Mo36948Tc457059Ru388867Sn784109Pr452160Sm361875Hg704477Pb745387Ra938844Np149791Am331048Bk429Fm32145Md301611Lr256845Unknown9124Unknown872703
Sb50283Pt16011At352775Bk98564
N83964F16193Ar3367Ti151452V30685Cu78400As482385Sr457374Rh138960Pd172740Sb241930I188763Xe66920Ba182997Sm84905Eu100776Hf62358Hg407624Pb337235At386854Rn154199Ra372408Ac244337Pa499726Am77469Bk546647Rf282551Sg195531Mt488877Ds54138Rg94381Nh465102Og9662Unknown239928
Ne175209Si426507P32944Fe18418Cu41802Rb288806Y29394Zr31837In197151Sb293065Pr249700Nd103296W219258Ac88526Pu2576Rf267875Sg101301Bh50554Nh84242Og360882Unknown131495Unknown218443
Si382270Y354753In236954Hs81135Og88166Unknown717489Unknown785672
Br256817Pd194189Tm186588Yb46604
Zr323142In114357Nd400115Fl123104Unknown298536Unknown396485
Ne224140Rb251645Zr594740In181225Nd258390Tb390434Ho2